## **1. Install the tool**
Clone the pLitter tool from Github and install it as python packe using pip

In [ ]:
!git clone https://github.com/gicait/pLitter.git
!pip install pLitter/.

Cloning into 'pLitter'...
remote: Enumerating objects: 2387, done.
remote: Counting objects: 100% (647/647), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 2387 (delta 469), reused 612 (delta 442), pack-reused 1740
Receiving objects: 100% (2387/2387), 124.71 MiB | 31.10 MiB/s, done.
Resolving deltas: 100% (1579/1579), done.
Processing ./pLitter
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 87 kB 4.0 MB/s 
     |████████████████████████████████| 68 kB 1.1 MB/s 
     |████████████████████████████████| 462 kB 39.2 MB/s 
  Created wheel for plitterstreet: filename=plitterstreet-1.0-p

## **2. Select your data**
Add your data to colab runtime, here we are downloading a sample data stored on google drive

A video file and GPS track file in CSV format is must. Check out the documentation for preparing it.

In [2]:
!mkdir data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nHRp1_0oik9fVsY0tmVuyarjuxPdXyDL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nHRp1_0oik9fVsY0tmVuyarjuxPdXyDL" -O data/sample.zip && rm -rf /tmp/cookies.txt
!unzip data/sample.zip -d data/.
!rm -rf 'data/sample.zip'

## **3. Import the tool**
We need two modules here, 

i. To convert the video to frames,

ii. To make predictions on the frames

In [ ]:
from plitterstreet.video import Video
from plitterstreet.plitter_street import StreetData

from plitterstreet.predict import EdgePredictor

## **4. Convert the street video to frames**

In [ ]:
vid = Video('data/sample_video.mp4', 'data/sample_GPS5.csv')
vid.ExtractbyGps()

Video is set and ready to make frames
GPS poingts found: 182
starting position set to: 52.68421052631579
destination path is not given, stores the iamges into a created folder with video file name in the directroy to input video
14.0155687 100.6425359 -9.387
saving image at 5110.368421052632
saving image at 6712.588235294119
saving image at 7744.578947368422
saving image at 15073.882352941177
saving image at 15721.588235294119
saving image at 16648.21052631579
saving image at 16964.31578947369
saving image at 17239.444444444438
saving image at 17684.333333333314
saving image at 18073.61111111111
saving image at 18740.944444444423
saving image at 19297.055555555547
saving image at 19630.722222222204
saving image at 25319.411764705874
Unable to read frame at 31031.0


'data/sample_video'

In [ ]:
street = StreetData('data/sample_video')
street.load_images_from_list()

Predictions file is invalid. Ignore it if the images are not predicted and saved.


In [ ]:
street.images

[{'latitude': 14.0155687,
  'longitude': 100.64253590000001,
  'name': 'sample_video_0.jpg'},
 {'latitude': 14.0155419,
  'longitude': 100.6425214,
  'name': 'sample_video_1.jpg'},
 {'latitude': 14.0155354,
  'longitude': 100.64252560000001,
  'name': 'sample_video_2.jpg'},
 {'latitude': 14.0155007,
  'longitude': 100.6425045,
  'name': 'sample_video_3.jpg'},
 {'latitude': 14.0153873,
  'longitude': 100.6425157,
  'name': 'sample_video_4.jpg'},
 {'latitude': 14.0153673,
  'longitude': 100.64251180000001,
  'name': 'sample_video_5.jpg'},
 {'latitude': 14.0153462,
  'longitude': 100.64251420000001,
  'name': 'sample_video_6.jpg'},
 {'latitude': 14.015339,
  'longitude': 100.64251630000001,
  'name': 'sample_video_7.jpg'},
 {'latitude': 14.0153315,
  'longitude': 100.64251780000001,
  'name': 'sample_video_8.jpg'},
 {'latitude': 14.0153172,
  'longitude': 100.64251850000001,
  'name': 'sample_video_9.jpg'},
 {'latitude': 14.0153026,
  'longitude': 100.64251730000001,
  'name': 'sample_vid

## **5. Run inference**

In [ ]:
predictor = EdgePredictor('/content/pLitter/models/automl_edge')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
street.predict_on_images(predictor, min_score_thr=0.25, dst_path='/content/data/sample_video')

In [ ]:
street.predictions

[{'box': [0.59, 0.57, 0.62, 0.6],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.59},
 {'box': [0.62, 0.75, 0.65, 0.82],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.54},
 {'box': [0.69, 0.87, 0.72, 0.93],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.52},
 {'box': [0.04, 0.58, 0.09, 0.65],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.5},
 {'box': [0.09, 0.76, 0.12, 0.78],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.49},
 {'box': [0.05, 0.31, 0.09, 0.34],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.4},
 {'box': [0.88, 0.94, 0.9, 0.98],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.36},
 {'box': [0.82, 0.3, 0.86, 0.32],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.31},
 {'box': [0.38, 0.81, 0.39, 0.83],
  'class': 'Plastic',
  'image': 'sample_video_0.jpg',
  'score': 0.28},
 {'box': [0.9, 0.67, 0.94, 0.72],

In [ ]:
street.save()

## **6. Plot the stats on the map**

In [ ]:
m = street.draw_street_on_map()
m

In [ ]:
m.save('/content/data/sample_video.html')